# California School Teacher Wages by County and District

## Load libraries and read in data

In [1]:
import geopandas as gpd
import pandas as pd
import folium
import numpy as np
from branca.element import Template, MacroElement
from folium.plugins import MeasureControl
from folium.plugins import geocoder

In [2]:
# County Shapefile (Polygon boudaries for map visual)
counties = gpd.read_file("/Users/nathanjones/Downloads/ca_counties/cnty19_1.shp")
# This is the table name in the CDE database for Teacher Salary Information based on Step / Column in Form J-90
tsal321 = gpd.read_file("/Users/nathanjones/Downloads/tsal321.csv")
# This is the table name in the CDE database for Teacher Salary Information Column Descriptions in Form J-90
tsal221 = gpd.read_file("/Users/nathanjones/Downloads/tsal221.csv")
# This is the table name in the CDE database for District Level Data
tsal121 = pd.read_csv("/Users/nathanjones/Downloads/tsal121.csv")

In [3]:
print("California Counties")
display(counties.head(3))
print("District Level Data")
display(tsal121.head(3))
print("Teacher Salary Education Level Descriptions")
display(tsal221.head(3))
print("Teacher Salary Data")
tsal321.head(3)

California Counties


,OBJECTID,COUNTY_NAM,COUNTY_ABB,COUNTY_NUM,COUNTY_COD,COUNTY_FIP,ISLAND,GlobalID,SHAPE_Leng,SHAPE_Area,geometry
0,1,Alameda,ALA,1,01,001,None,{E6F92268-D2DD-4CFB-8B79-5B4B2F07C559},308998.650975,-3.402787e+09,"POLYGON ((-13611173.233 4566017.932, -13611061..."
1,2,Alpine,ALP,2,02,003,None,{870479B2-480A-494B-8352-AD60578839C1},274888.492662,-3.146939e+09,"POLYGON ((-13312326.921 4680816.110, -13312311..."
2,3,Amador,AMA,3,03,005,None,{4F45B3A6-BE10-461C-8945-6B2AAA7119F6},361708.438649,-2.562635e+09,"POLYGON ((-13366405.546 4679183.529, -13366408..."


District Level Data


,county,district,cds,ts1_dname,ts1_county,ts1_type,ts1_ada,ts1_bacol,ts1_pctchg,ts1_ndays,...,chtfte,r_other,ur_other,retire,uretire,bonusapp,liability,studydate,ba30col,tsal1_ts
0,1,61143,161143,BERKELEY UNIFIED,Alameda,4,9426.030273,5,9.50,180,...,0.0,0,0,1,1,N,23348827.0,06/30/2018,2,2021-10-21 13:06:58
1,1,61150,161150,CASTRO VALLEY UNIFIED,Alameda,4,8976.769531,4,3.85,180,...,0.0,0,0,0,1,N,3295764.0,06/30/2021,2,2021-10-29 14:25:01
2,1,61176,161176,FREMONT UNIFIED,Alameda,4,34400.540009,3,0.56,180,...,20.7,0,0,0,0,Y,128587799.0,6/16/2021,1,2021-07-21 16:20:14


Teacher Salary Education Level Descriptions


,county,district,cds,ts2_col,ts2_col1,ts2_col1a,ts2_col2,ts2_col3,ts2_col3a,ts2_id,geometry
0,01,10017,0110017,1,EMERG/,INTERN,,,,3594,None
1,01,10017,0110017,2,PRELIM,AND,CLEAR,,,3595,None
2,01,61119,0161119,1,BA,,,,,1582,None


Teacher Salary Data


,county,district,cds,ts3_col,ts3_step,ts3_salary,ts3_fte,ts3_salcos,ts3_id,geometry
0,01,10017,0110017,1,1,66441,0,0,166648,None
1,01,10017,0110017,1,2,69140,0,0,166649,None
2,01,10017,0110017,1,3,71976,0.67,48223.920000000006,166650,None


## Data Cleaning and light exploration

We can clean up part of the shapefile to remove the portions of land which are islands off the mainland US. In addition, we will need to clean up some of these strings in order to join our tables together.

Our `tsal321` and `tsal221` tables contain a leading 0 in their county numbers, so we need to pad the county number from our `counties` table to join all 3 of these up.

We also need to rename our columns to give them their real world meaning (check the database **.readme**). To do this we need to combine the education level description level columns `ts2_col1`, `ts2_col1a`, `ts2_col2` from the `tsal221` table which describe the education level for the column number given in the `ts3_col` column from the `tsal321` table.

I found the database documentation surrounding the years of experience and education level (step & column) to be difficult to understand. In my opinion, it wasn't easily clear how to answer the following question:

`Given a teacher's years of applicable experience and their education level, return the exact salary for that teacher at each district`

So, I reached out for the help to SACSINFO@cde.ca.gov and was given confirmation that this information was correct and joining these tables was the way to access the data I wanted (to the best of the government worker's knowledge).

**NOTE**: They did mention something weird I noticed with the Browns Elementary (CDS 5171365). They submitted multiple salaries under the same years of experience and education level. They were ommitted from this as a result.

In [4]:
# Clean up counties table to exclude Island portions off the mainland USA
counties = counties[counties["ISLAND"].isnull()]
counties["COUNTY_NUM"] = counties["COUNTY_NUM"].astype(str).str.rjust(2, '0')

In [5]:
teacher_salary = tsal321[tsal321["cds"] != '5171365'].merge(tsal221[tsal221["cds"] != '5171365'], how = "left", left_on = ["cds", "ts3_col"], right_on = ["cds", "ts2_col"])
new_column_names = ["county", "district", "cds", "education_level_column", "education_level_desc_1","education_level_desc_2", "education_level_desc_3", "years_experience", "salary"]
teacher_salary = teacher_salary[["county_x", "district_x", "cds", "ts3_col", "ts2_col1", "ts2_col1a", "ts2_col2", "ts3_step", "ts3_salary", ]]
teacher_salary.columns = new_column_names
county_salary = counties.merge(teacher_salary, how = 'left', left_on = 'COUNTY_NUM', right_on = "county")
print("Combined Teacher Salary Data from Form J90")
display(teacher_salary.head(3))
print("County Teacher Salary Information")
display(county_salary.head(3))
county_salary["education_level_desc"] = county_salary[["education_level_desc_1", "education_level_desc_2", "education_level_desc_3"]].astype(str).agg(' '.join, axis=1)
county_salary.drop(columns = ["education_level_desc_1", "education_level_desc_2", "education_level_desc_3"], inplace = True)


Combined Teacher Salary Data from Form J90


,county,district,cds,education_level_column,education_level_desc_1,education_level_desc_2,education_level_desc_3,years_experience,salary
0,01,10017,0110017,1,EMERG/,INTERN,,1,66441
1,01,10017,0110017,1,EMERG/,INTERN,,2,69140
2,01,10017,0110017,1,EMERG/,INTERN,,3,71976


County Teacher Salary Information


,OBJECTID,COUNTY_NAM,COUNTY_ABB,COUNTY_NUM,COUNTY_COD,COUNTY_FIP,ISLAND,GlobalID,SHAPE_Leng,SHAPE_Area,geometry,county,district,cds,education_level_column,education_level_desc_1,education_level_desc_2,education_level_desc_3,years_experience,salary
0,1,Alameda,ALA,01,01,001,None,{E6F92268-D2DD-4CFB-8B79-5B4B2F07C559},308998.650975,-3.402787e+09,"POLYGON ((-13611173.233 4566017.932, -13611061...",01,10017,0110017,1,EMERG/,INTERN,,1,66441
1,1,Alameda,ALA,01,01,001,None,{E6F92268-D2DD-4CFB-8B79-5B4B2F07C559},308998.650975,-3.402787e+09,"POLYGON ((-13611173.233 4566017.932, -13611061...",01,10017,0110017,1,EMERG/,INTERN,,2,69140
2,1,Alameda,ALA,01,01,001,None,{E6F92268-D2DD-4CFB-8B79-5B4B2F07C559},308998.650975,-3.402787e+09,"POLYGON ((-13611173.233 4566017.932, -13611061...",01,10017,0110017,1,EMERG/,INTERN,,3,71976


Now that we have our joined up tables, we have a starting point.

For the county level view, we filter out the all rows except for positions with

In [6]:
df = county_salary[(county_salary["years_experience"] == '1') & (county_salary["education_level_desc"].str.contains('ba|ma', case= False))]

df['education_level_column'] = df['education_level_column'].astype(int)
df['salary'] = df['salary'].astype(float)

/Users/nathanjones/opt/anaconda3/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [7]:
county_avg_salary = df.groupby(["COUNTY_NAM"]).mean()[["salary"]]
county_avg_salary = county_avg_salary.sort_values(by = "salary", ascending = False).reset_index().reset_index().rename(columns = {"index" : "salary_rank"})

In [8]:
county_df = counties.merge(county_avg_salary, how = 'left', on = ["COUNTY_NAM"])
county_df["COUNTY_NAM"] = county_df["COUNTY_NAM"] + " County"
county_df["salary_rank"] = county_df["salary_rank"] + 1

In [9]:
# 3 Highest Salary Counties
county_df.sort_values(by = "salary", ascending = False).head(3)

,OBJECTID,COUNTY_NAM,COUNTY_ABB,COUNTY_NUM,COUNTY_COD,COUNTY_FIP,ISLAND,GlobalID,SHAPE_Leng,SHAPE_Area,geometry,salary_rank,salary
27,28,Napa County,NAP,28,28,055,None,{630FF402-4AFD-4950-93B9-773E16DAB8DC},356598.937630,-3.348149e+09,"POLYGON ((-13624348.918 4701112.763, -13624335...",1.0,71597.520000
42,43,Santa Clara County,SCL,43,43,085,None,{2220F7B9-9361-4FB8-9456-872109CF6CF7},430617.481386,-5.341596e+09,"POLYGON ((-13524557.408 4506504.112, -13522201...",2.0,66970.627451
40,41,San Mateo County,SMT,41,41,081,None,{B4F387B3-07F1-43A1-B18A-A6D81C17E28B},296746.199118,-2.272024e+09,"POLYGON ((-13624775.603 4538293.155, -13622844...",3.0,64520.390071


In [10]:
county_df = county_df[~(county_df["salary"]).isnull()]
# Make salary more readable
county_df["salary_formated"] = county_df["salary"].apply(lambda x: '${:,.2f}'.format(x))

In [11]:
m = folium.Map(location=[37.411292, -118], zoom_start= 6)
# Set up Bins for number of drivers
bins = np.linspace(county_df["salary"].min(), county_df["salary"].max(), 8)
# Define Pallete for Choropleth
choropleth_colors = np.array(['#fff7ec', '#fee8c8', '#fdd49e', '#fdbb84', '#fc8d59', '#ef6548', '#d7301f', '#990000'])
# Polygon Style Function

def style_function(feature):
    sal = feature['properties']['salary']
    return {'color':'black', 
            'fillOpacity': .8,
            'weight': 1,
            'fillColor':  
            '#d9d9d9' 
                if sal == np.nan
                else choropleth_colors[0] if sal >= bins[0] and sal < bins[1]
                else choropleth_colors[1] if sal >= bins[1] and sal < bins[2]
                else choropleth_colors[2] if sal >= bins[2] and sal < bins[3]
                else choropleth_colors[3] if sal >= bins[3] and sal < bins[4]
                else choropleth_colors[4] if sal >= bins[4] and sal < bins[5]
                else choropleth_colors[5] if sal >= bins[5] and sal < bins[6]
                else choropleth_colors[6] if sal >= bins[6] and sal < bins[7]
                else choropleth_colors[7] if sal >= bins[7]
                else 'black'}

highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}

folium.GeoJson(
    data = county_df,
    style_function=style_function,
    highlight_function=highlight_function,
    name= "Avg. BA/MA Salary",
    overlay=True,
    control=True,
    show=True,
    smooth_factor=None,
    zoom_on_click= True,
    tooltip= folium.features.GeoJsonTooltip(
        fields=['COUNTY_NAM',"salary_formated"],
        aliases=['Name:',"Avg. Salary:"],
        style = """
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 2px,
        box-shadow: 3px; 
        """)
).add_to(m)


####################################### Adding in Manual Legend #######################################

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Avg. Salary for BA/MA Teachers by County</div>
<div class='legend-scale'>
  <ul class='legend-labels', style="font-weight: bold;">
    <li><span style='background:#fff7ec;opacity:0.9;'></span>""" + str('${:,.2f}'.format(bins[0])) + """ - """ + str('${:,.2f}'.format(bins[1])) + """</li>
    <li><span style='background:#fee8c8;opacity:0.9;'></span>""" + str('${:,.2f}'.format(bins[1])) + """ - """ + str('${:,.2f}'.format(bins[2])) + """</li>
    <li><span style='background:#fdd49e;opacity:0.9;'></span>""" + str('${:,.2f}'.format(bins[2])) + """ - """ + str('${:,.2f}'.format(bins[3])) + """</li>
    <li><span style='background:#fdbb84;opacity:0.9;'></span>""" + str('${:,.2f}'.format(bins[3])) + """ - """ + str('${:,.2f}'.format(bins[4])) + """</li>
    <li><span style='background:#fc8d59;opacity:0.9;'></span>""" + str('${:,.2f}'.format(bins[4])) + """ - """ + str('${:,.2f}'.format(bins[5])) + """</li>
    <li><span style='background:#ef6548;opacity:0.9;'></span>""" + str('${:,.2f}'.format(bins[5])) + """ - """ + str('${:,.2f}'.format(bins[6])) + """</li>
    <li><span style='background:#d7301f;opacity:0.9;'></span>""" + str('${:,.2f}'.format(bins[6])) + """ - """ + str('${:,.2f}'.format(county_df['salary'].max())) + """</li>

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

loc = 'Average Salary by County for a 1st Year Teacher with a Masters or Bachelors'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().add_child(macro)
m.add_child(MeasureControl(position = 'bottomleft', primary_length_unit='miles', secondary_length_unit='meters', primary_area_unit='sqmiles', secondary_area_unit=np.nan))
folium.plugins.Geocoder().add_to(m)

m.save("/Users/nathanjones/Downloads/ca_teacher_salary_by_county.html")

## Read in District Level Data

We can repeat the process with the district level data by joining on CDS, a identifier that combines the county id and the district id together.

In [12]:
districts = gpd.read_file("/Users/nathanjones/Downloads/ca_school_districts/California_School_District_Areas_2020-21.shp")
districts.rename(columns = {"CDCode" : 'cds'}, inplace = True)
districts.head()

,OBJECTID,Year,FedID,cds,CDSCode,CountyName,DistrictNa,DistrictTy,GradeLow,GradeHigh,...,SWDcount,SWDpct,SEDcount,SEDpct,FRPMcount,FRPMpct,DistrctAre,Shape__Are,Shape__Len,geometry
0,1,2020-21,0601770,0161119,01611190000000,Alameda,Alameda Unified,Unified,PK,12,...,1275,11.7,3202,29.3,3101,28.4,11.455570,4.755489e+07,56522.982683,"MULTIPOLYGON (((-13606222.820 4540862.699, -13..."
1,2,2020-21,0601860,0161127,01611270000000,Alameda,Albany City Unified,Unified,KG,12,...,311,8.9,608,17.4,587,16.8,1.703654,7.096327e+06,12696.382797,"POLYGON ((-13612893.866 4565099.707, -13612896..."
2,3,2020-21,0604740,0161143,01611430000000,Alameda,Berkeley Unified,Unified,PK,12,...,1091,11.6,2563,27.2,2431,25.8,10.483362,4.364648e+07,43695.341538,"POLYGON ((-13609482.480 4565074.597, -13609483..."
3,4,2020-21,0607800,0161150,01611500000000,Alameda,Castro Valley Unified,Unified,PK,12,...,850,9.2,2067,22.5,1977,21.5,68.442604,2.838285e+08,142492.767565,"MULTIPOLYGON (((-13581539.054 4528172.150, -13..."
4,5,2020-21,0612630,0161168,01611680000000,Alameda,Emery Unified,Unified,PK,12,...,69,9.9,561,80.4,555,79.5,1.289416,5.363392e+06,13741.272894,"POLYGON ((-13613999.038 4555592.769, -13614126..."


In [13]:
# Add leading 0 to cds
tsal121["cds"] = tsal121["cds"].astype(str).str.rjust(7, '0')
# What kind of school
def f(col_type):
    if col_type == 0:
        return 'County Office of Education'
    elif col_type == 1:
        return 'Elementary'
    elif col_type == 2:
        return 'High School'
    elif col_type == 3:
        return 'Common Admin District'
    elif col_type == 4:
        return 'Unified'

tsal121["ts1_type"] = tsal121["ts1_type"].apply(f)

Here we need to narrow in on applicable salaries based on the column descriptions. Unfortunately, each school districts Salary and Benefits Schedule for the Certificated Bargaining Unit (Form J-90) can have a number of different education level columns.

Say we want to filter out for a specific degree, to really demonstrate how we can identify a teachers salary **given their years of experience and education level.**

For this example, we will look at teachers with the lowest level of experience (listed as step 1) with a masters degree. Since the data is not in a standardized format, we will need to wrangle and pattern match some of this text data to retrieve the information we want.

In [14]:
# Which CDS's are missing
print(f'{len(list(set(districts["cds"]) - set(tsal121["cds"])))} Missing CDS codes from the school districts shapefile')
district_salary = districts.merge(tsal121, how = "left", on = "cds").merge(teacher_salary, how = "left", on = "cds")

print("District Teacher Salary Information")
display(district_salary.head(3))
district_salary["education_level_desc"] = district_salary[["education_level_desc_1", "education_level_desc_2", "education_level_desc_3"]].astype(str).agg(' '.join, axis=1)
district_salary.drop(columns = ["education_level_desc_1", "education_level_desc_2", "education_level_desc_3"], inplace = True)
df = district_salary[(district_salary["years_experience"] == '1') & (district_salary["education_level_desc"].str.contains('ba|ma', case= False))]

df['education_level_column'] = df['education_level_column'].astype(int)
df['salary'] = df['salary'].astype(float)


187 Missing CDS codes from the school districts shapefile
District Teacher Salary Information


,OBJECTID,Year,FedID,cds,CDSCode,CountyName,DistrictNa,DistrictTy,GradeLow,GradeHigh,...,ba30col,tsal1_ts,county_y,district_y,education_level_column,education_level_desc_1,education_level_desc_2,education_level_desc_3,years_experience,salary
0,1,2020-21,0601770,0161119,01611190000000,Alameda,Alameda Unified,Unified,PK,12,...,2.0,2021-06-29 13:22:59,01,61119,1,BA,,,1,58975
1,1,2020-21,0601770,0161119,01611190000000,Alameda,Alameda Unified,Unified,PK,12,...,2.0,2021-06-29 13:22:59,01,61119,1,BA,,,2,59555
2,1,2020-21,0601770,0161119,01611190000000,Alameda,Alameda Unified,Unified,PK,12,...,2.0,2021-06-29 13:22:59,01,61119,1,BA,,,3,60135


/Users/nathanjones/opt/anaconda3/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


### To keep things simple, we won't make our filters perfect.

**We can use the following rules**:
* 1 year of experience
* Anything that includes a BA
* Anything that inlcudes an MA, unless it is of the form $MA+UNITS$

In [15]:
import re
first_year = district_salary[district_salary["years_experience"] == '1']
first_year["education_level_desc"] = first_year["education_level_desc"].str.strip()
# 4190 Possible Salaries to sift through
print(first_year.shape)


# No ma+{units} unless there is a ba as well
indexes = first_year[(first_year["education_level_desc"].str.contains('ba\+\d+|^ma|\+ma| *ma', case = False)) & ~(first_year["education_level_desc"].str.contains('ma\+| *ma\+', case = False))].index.values

df = first_year[first_year.index.isin(indexes)].reset_index(drop = True)
df["salary"] = df["salary"].astype(int)
df.head()

(4190, 166)


,OBJECTID,Year,FedID,cds,CDSCode,CountyName,DistrictNa,DistrictTy,GradeLow,GradeHigh,...,liability,studydate,ba30col,tsal1_ts,county_y,district_y,education_level_column,years_experience,salary,education_level_desc
0,1,2020-21,0601770,0161119,01611190000000,Alameda,Alameda Unified,Unified,PK,12,...,16456178.0,6/30/2020,2.0,2021-06-29 13:22:59,01,61119,2,1,59555,BA+30
1,1,2020-21,0601770,0161119,01611190000000,Alameda,Alameda Unified,Unified,PK,12,...,16456178.0,6/30/2020,2.0,2021-06-29 13:22:59,01,61119,3,1,60798,BA+45
2,1,2020-21,0601770,0161119,01611190000000,Alameda,Alameda Unified,Unified,PK,12,...,16456178.0,6/30/2020,2.0,2021-06-29 13:22:59,01,61119,4,1,64548,BA+60
3,1,2020-21,0601770,0161119,01611190000000,Alameda,Alameda Unified,Unified,PK,12,...,16456178.0,6/30/2020,2.0,2021-06-29 13:22:59,01,61119,5,1,68471,BA+75
4,2,2020-21,0601860,0161127,01611270000000,Alameda,Albany City Unified,Unified,KG,12,...,22005621.0,06/30/2020,1.0,2021-06-08 15:07:18,01,61127,1,1,51344,BA<60 OR MA


In [16]:
district_avg_salary = df.groupby(["DistrictNa"]).mean()[["salary"]]
district_avg_salary = district_avg_salary.sort_values(by = "salary", ascending = False).reset_index().reset_index().rename(columns = {"index" : "salary_rank"})
district_avg_salary


,salary_rank,DistrictNa,salary
0,0,Saint Helena Unified,90378.400000
1,1,Mountain View-Los Altos Union High,90066.250000
2,2,Woodside Elementary,84926.750000
3,3,Fremont Union High,82104.666667
4,4,Santa Clara Unified,81274.666667
...,...,...,...
731,731,Surprise Valley Joint Unified,37818.250000
732,732,Ravendale-Termo Elementary,37709.000000
733,733,Caliente Union Elementary,36409.500000
734,734,Peninsula Union,36262.666667


In [17]:
district_df = districts.merge(district_avg_salary, how = 'left', on = ["DistrictNa"])
district_df["DistrictNa"] = district_df["DistrictNa"] + " School District"
district_df["salary_rank"] = district_df["salary_rank"] + 1
district_df["salary"] = district_df["salary"].fillna(0)
district_df["salary_formated"] = district_df["salary"].apply(lambda x: '${:,.2f}'.format(x))

In [18]:
# 3 Highest Salary Districts
district_df.sort_values(by = "salary", ascending = False).head(3)

,OBJECTID,Year,FedID,cds,CDSCode,CountyName,DistrictNa,DistrictTy,GradeLow,GradeHigh,...,SEDpct,FRPMcount,FRPMpct,DistrctAre,Shape__Are,Shape__Len,geometry,salary_rank,salary,salary_formated
417,418,2020-21,0637830,2866290,28662900000000,Napa,Saint Helena Unified School District,Unified,KG,12,...,44.8,470,39.6,313.582382,1.331788e+09,397901.014139,"POLYGON ((-13597600.245 4667406.460, -13597423...",1.0,90378.40,"$90,378.40"
681,682,2020-21,0626310,4369609,43696090000000,Santa Clara,Mountain View-Los Altos Union High School Dist...,High,09,12,...,15.6,533,11.7,36.818411,1.512952e+08,85341.725158,"POLYGON ((-13589925.214 4502275.972, -13589657...",2.0,90066.25,"$90,066.25"
640,641,2020-21,0643140,4169088,41690880000000,San Mateo,Woodside Elementary School District,Elementary,KG,08,...,11.1,40,10.8,11.476322,4.721103e+07,38254.334314,"POLYGON ((-13608079.069 4499963.511, -13608118...",3.0,84926.75,"$84,926.75"


In [20]:
m = folium.Map(location=[37.5, -117], zoom_start= 6)
# Set up Bins for number of drivers
bins = np.linspace(district_df["salary"].min(), district_df["salary"].max(), 8).round(2)
# Define Pallete for Choropleth
choropleth_colors = np.array(['#fff7ec', '#fee8c8', '#fdd49e', '#fdbb84', '#fc8d59', '#ef6548', '#d7301f', '#990000'])
# Polygon Style Function

def style_function(feature):
    sal = feature['properties']['salary']
    return {'color':'black', 
            'fillOpacity': .8,
            'weight': 1,
            'fillColor':  
            '#d9d9d9' 
                if sal == 0
                else choropleth_colors[0] if sal >= bins[0] and sal < bins[1]
                else choropleth_colors[1] if sal >= bins[1] and sal < bins[2]
                else choropleth_colors[2] if sal >= bins[2] and sal < bins[3]
                else choropleth_colors[3] if sal >= bins[3] and sal < bins[4]
                else choropleth_colors[4] if sal >= bins[4] and sal < bins[5]
                else choropleth_colors[5] if sal >= bins[5] and sal < bins[6]
                else choropleth_colors[6] if sal >= bins[6] and sal < bins[7]
                else choropleth_colors[7] if sal >= bins[7]
                else 'black'}

highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}

folium.GeoJson(
    data = district_df,
    style_function=style_function,
    highlight_function=highlight_function,
    name= "Salary Per District for BA/MA 1st Year Positions",
    overlay=True,
    control=True,
    show=True,
    smooth_factor=None,
    zoom_on_click= True,
    tooltip= folium.features.GeoJsonTooltip(
        fields=['DistrictNa',"salary_formated"],
        aliases=['Name:',"Avg. Salary:"],
        style = """
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 2px,
        box-shadow: 3px; 
        """)
).add_to(m)


# NIL = folium.features.GeoJson(
#     data = df,
#     style_function=style_function, 
#     control=False,
#     zoom_on_click = True,
#     highlight_function=highlight_function, 
#     tooltip=folium.features.GeoJsonTooltip(
#         fields=['COUNTY_NAM',"salary"],
#         aliases=['Name:',"Avg. Salary:"],
#         style = """
#         background-color: #d9d9d9;
#         border: 2px solid black;
#         border-radius: 2px,
#         box-shadow: 3px; 
#         """,
#     max_width = 500,
#     sticky = True,
#     labels = True,
#     show = True
# ))
# m.add_child(NIL)
# m.keep_in_front(NIL)

####################################### Adding in Manual Legend #######################################

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Avg. Salary for BA/MA Teachers by School District</div>
<div class='legend-scale'>
  <ul class='legend-labels', style="font-weight: bold;">
  <li><span style='background:#d9d9d9;opacity:0.8;'></span>Missing data represented as $ 0</li>
    <li><span style='background:#fff7ec;opacity:0.9;'></span>""" + str('${:,.2f}'.format(bins[0])) + """ - """ + str('${:,.2f}'.format(bins[1])) + """</li>
    <li><span style='background:#fee8c8;opacity:0.9;'></span>""" + str('${:,.2f}'.format(bins[1])) + """ - """ + str('${:,.2f}'.format(bins[2])) + """</li>
    <li><span style='background:#fdd49e;opacity:0.9;'></span>""" + str('${:,.2f}'.format(bins[2])) + """ - """ + str('${:,.2f}'.format(bins[3])) + """</li>
    <li><span style='background:#fdbb84;opacity:0.9;'></span>""" + str('${:,.2f}'.format(bins[3])) + """ - """ + str('${:,.2f}'.format(bins[4])) + """</li>
    <li><span style='background:#fc8d59;opacity:0.9;'></span>""" + str('${:,.2f}'.format(bins[4])) + """ - """ + str('${:,.2f}'.format(bins[5])) + """</li>
    <li><span style='background:#ef6548;opacity:0.9;'></span>""" + str('${:,.2f}'.format(bins[5])) + """ - """ + str('${:,.2f}'.format(bins[6])) + """</li>
    <li><span style='background:#d7301f;opacity:0.9;'></span>""" + str('${:,.2f}'.format(bins[6])) + """ - """ + str('${:,.2f}'.format(district_df['salary'].max())) + """</li>

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

m.get_root().add_child(macro)
m.add_child(MeasureControl(position = 'bottomleft', primary_length_unit='miles', secondary_length_unit='meters', primary_area_unit='sqmiles', secondary_area_unit=np.nan))
folium.plugins.Geocoder().add_to(m)
loc = 'Average District Salary for a 1st Year Teacher with a Masters (or Bachelors with significant units)'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)
m.get_root().html.add_child(folium.Element(title_html))

m.save("/Users/nathanjones/Downloads/ca_teacher_salary_by_district.html")
